In [26]:
%load_ext autoreload
%autoreload 2

import lightgbm as lgb
import numpy as np
import sklearn as sk
import pandas as pd

from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTENC
import pickle
import json
import xgboost as xgb
import os

# grid search
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold

from skorch import NeuralNetRegressor


from tqdm.notebook import tqdm


# Neural Networks

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from utils import save_result, compute_sample_weights, compute_kl_divergence, load_hyperparams, save_hyperparams

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
data_type = 'slope_removed'
data = pd.read_pickle(f'data/{data_type}/train.pkl')

test_data = pd.read_pickle(f'data/{data_type}/test.pkl')

# Drop and store the ID column
test_ids = test_data['ID']
test_data = test_data.drop('ID', axis=1)

# Drop the ID column
data = data.drop('ID', axis=1)

# One hot encode train and test data
# Only affect categorical columns
data = pd.get_dummies(data, drop_first=True)
test_data = pd.get_dummies(test_data, drop_first=True)

# 80% train, 20% test. Target is target
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.2, stratify=data['work_hours_1'])

# iso = IsolationForest(contamination=0.01)
# yhat = iso.fit_predict(X_train)
# mask = yhat != -1
# X_train, y_train = X_train[mask], y_train[mask]

best_params_lgb = load_hyperparams('lgb', data_type)
best_params_svr = load_hyperparams('svr', data_type)
best_params_mlp = load_hyperparams('mlp', data_type)
best_params_knn = load_hyperparams('knn', data_type)
best_params_lr = load_hyperparams('lr', data_type)

# Use lightgbm
model1 = lgb.LGBMRegressor(
    **best_params_lgb,
    verbose=-1
)

model_svr = SVR(**best_params_svr)
model_mlp = MLPRegressor(
    **best_params_mlp
)

model_knn = KNeighborsRegressor(
    **best_params_knn
)

estimators = [
    ('lgb', model1),
    ('svr', model_svr),
    ('mlp', model_mlp),
    ('knn', model_knn),
]

meta_model = LinearRegression(**best_params_lr)

model = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_model,
    passthrough=True,
)

model.fit(X_train, y_train)

result = model.predict(X_test)
mae = mean_absolute_error(y_test, result)
print(f'MAE: {mae}')

save_result(mae, data_type)

test_result = model.predict(test_data)

# Save predictions to predictions/data_type.csv
# There must be only be an ID and a target column
pd.DataFrame({'ID': test_ids, 'target': test_result}).to_csv(f'predictions/{data_type}.csv', index=False)

Loaded best hyperparameters for lgb from config/slope_removed/best_params_lgb.json.
Loaded best hyperparameters for svr from config/slope_removed/best_params_svr.json.
Loaded best hyperparameters for mlp from config/slope_removed/best_params_mlp.json.
Loaded best hyperparameters for knn from config/slope_removed/best_params_knn.json.
No existing hyperparameter file for lr. Proceeding with search.
MAE: 2.1692718099691732


## Exhaustive grid search

In [30]:
# Helper function to load hyperparameters from JSON if available
NUM_JOBS = 8
VERBOSE = 3

# Load data
data_type = 'slope_removed'
data = pd.read_pickle(f'data/{data_type}/train.pkl')
test_data = pd.read_pickle(f'data/{data_type}/test.pkl')

# Store and drop the ID column
test_ids = test_data['ID']
test_data = test_data.drop('ID', axis=1)
data = data.drop('ID', axis=1)

# One-hot encode train and test data with drop_first=True
data = pd.get_dummies(data, drop_first=True)
test_data = pd.get_dummies(test_data, drop_first=True)

# Ensure both train and test have the same columns after one-hot encoding
# Align the test data to have the same columns as training data
test_data = test_data.reindex(columns=data.columns.drop('target'), fill_value=0)

# Define features and target
X = data.drop('target', axis=1)
y = data['target']

# Stratify based on 'work_hours_1' by using it in initial train/test split
# This ensures that the distribution of 'work_hours_1' is maintained
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.1,
    stratify=data['work_hours_1'],
    random_state=42
)

# Define hyperparameter grids with regularization
param_grid_lgb = {
    'num_leaves': [31, 50, 70, 100],
    'max_depth': [-1, 10, 20, 50],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 500],
    'reg_alpha': [0, 0.1, 1, 10],
    'reg_lambda': [0, 0.1, 1, 10]
}

param_grid_svr = {
    'kernel': ['rbf', 'linear'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto'],
    'epsilon': [0.1, 0.2, 0.5, 1],
}

param_grid_mlp = {
    'hidden_layer_sizes': [(50, 50), (100, 100), (100, 50, 100), (100, 100, 100), (100, 100, 100, 100)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [500, 1000, 2000],
    'early_stopping': [True],
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
}

# Initialize GridSearchCV for each model with KFold CV
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Optimize LightGBM Regressor
print("Optimizing LightGBM Regressor...")
best_params_lgb = load_hyperparams('lgb', data_type)

model_lgb = lgb.LGBMRegressor(verbose=-1, **best_params_lgb)

if not best_params_lgb:
    model_lgb = lgb.LGBMRegressor(verbose=-1)
    grid_search_lgb = GridSearchCV(
        estimator=model_lgb,
        param_grid=param_grid_lgb,
        scoring='neg_mean_absolute_error',
        cv=cv,
        n_jobs=NUM_JOBS,
        verbose=VERBOSE
    )
    grid_search_lgb.fit(X_train, y_train)
    best_lgb = grid_search_lgb.best_estimator_
    best_params_lgb = grid_search_lgb.best_params_
    save_hyperparams('lgb', data_type, best_params_lgb)
    model_lgb = best_lgb

# Optimize SVR
print("Optimizing SVR...")
best_params_svr = load_hyperparams('svr', data_type)

model_svr = SVR(**best_params_svr)

if not best_params_svr:
    grid_search_svr = GridSearchCV(
        estimator=model_svr,
        param_grid=param_grid_svr,
        scoring='neg_mean_absolute_error',
        cv=cv,
        n_jobs=NUM_JOBS,
        verbose=VERBOSE
    )
    grid_search_svr.fit(X_train, y_train)
    best_svr = grid_search_svr.best_estimator_
    best_params_svr = grid_search_svr.best_params_
    save_hyperparams('svr', data_type, best_params_svr)
    model_svr = best_svr

# Optimize MLP Regressor
print("Optimizing MLP Regressor...")
best_params_mlp = load_hyperparams('mlp', data_type)

model_mlp = MLPRegressor(**best_params_mlp)

if not best_params_mlp:
    grid_search_mlp = GridSearchCV(
        estimator=model_mlp,
        param_grid=param_grid_mlp,
        scoring='neg_mean_absolute_error',
        cv=cv,
        n_jobs=NUM_JOBS,
        verbose=VERBOSE
    )
    grid_search_mlp.fit(X_train, y_train)
    best_mlp = grid_search_mlp.best_estimator_
    best_params_mlp = grid_search_mlp.best_params_
    save_hyperparams('mlp', data_type, best_params_mlp)
    model_mlp = best_mlp
    
# Optimize KNN Regressor
print("Optimizing KNN Regressor...")
best_params_knn = load_hyperparams('knn', data_type)

model_knn = KNeighborsRegressor(**best_params_knn)

if not best_params_knn:
    grid_search_knn = GridSearchCV(
        estimator=model_knn,
        param_grid=param_grid_knn,
        scoring='neg_mean_absolute_error',
        cv=cv,
        n_jobs=NUM_JOBS,
        verbose=VERBOSE
    )
    grid_search_knn.fit(X_train, y_train)
    best_knn = grid_search_knn.best_estimator_
    best_params_knn = grid_search_knn.best_params_
    save_hyperparams('knn', data_type, best_params_knn)
    model_knn = best_knn

# Build estimators with optimized models
estimators = [
    ('lgb', model_lgb),
    ('svr', model_svr),
    ('mlp', model_mlp),
    ('knn', model_knn),
]

# Update param_grid_meta for ElasticNet
param_grid_meta = {
    'final_estimator__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],
    'final_estimator__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
    'final_estimator__fit_intercept': [True, False],
    'final_estimator__max_iter': [1000, 2000],
    'final_estimator__tol': [1e-4, 1e-3, 1e-2],
    'final_estimator__selection': ['cyclic', 'random'],
    'final_estimator__random_state': [42]
}

# Initialize stacking regressor with ElasticNet
stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=ElasticNet(),
    passthrough=True,
    n_jobs=NUM_JOBS
)

# Optimize meta-estimator
print("Optimizing meta-estimator...")
best_params_meta = load_hyperparams('lr', data_type)

if best_params_meta:
    meta_estimator = ElasticNet(**best_params_meta)
    stacking_regressor = StackingRegressor(
        estimators=estimators,
        final_estimator=meta_estimator,
        passthrough=True,
        n_jobs=NUM_JOBS
    )
    print("Fitting the final model on all data...")
    stacking_regressor.fit(X, y)
else:
    grid_search_meta = GridSearchCV(
        estimator=stacking_regressor,
        param_grid=param_grid_meta,
        scoring='neg_mean_absolute_error',
        cv=cv,
        n_jobs=1,
        verbose=VERBOSE
    )
    grid_search_meta.fit(X_train, y_train)
    best_stacking = grid_search_meta.best_estimator_
    best_params_meta = grid_search_meta.best_params_
    best_params_meta = {k.replace('final_estimator__', ''): v for k, v in best_params_meta.items()}
    save_hyperparams('lr', data_type, best_params_meta)
    stacking_regressor = best_stacking

# Calculate MAE on validation set
if best_params_meta:
    print("Calculating MAE on validation set...")
    # if 'best_stacking' in locals():
    #     y_val_pred = best_stacking.predict(X_val)
    # else:
    y_val_pred = stacking_regressor.predict(X_val)
    mae_val = mean_absolute_error(y_val, y_val_pred)
    print(f'Validation MAE: {mae_val}')

# Save the final stacked model
with open('final_model.pkl', 'wb') as f:
    pickle.dump(stacking_regressor, f)
    print("Final stacked model saved to 'final_model.pkl'.")

# Generate predictions on test data
print("Generating predictions on test data...")
predictions = stacking_regressor.predict(test_data)

# Save predictions to CSV
output_filepath = f'predictions/tuned_{data_type}.csv'
pd.DataFrame({'ID': test_ids, 'target': predictions}).to_csv(output_filepath, index=False)
print(f'Predictions saved to {output_filepath}')

Optimizing LightGBM Regressor...
Loaded best hyperparameters for lgb from config/slope_removed/best_params_lgb.json.
Optimizing SVR...
Loaded best hyperparameters for svr from config/slope_removed/best_params_svr.json.
Optimizing MLP Regressor...
Loaded best hyperparameters for mlp from config/slope_removed/best_params_mlp.json.
Optimizing KNN Regressor...
Loaded best hyperparameters for knn from config/slope_removed/best_params_knn.json.
Optimizing meta-estimator...
No existing hyperparameter file for lr. Proceeding with search.
Fitting 5 folds for each of 600 candidates, totalling 3000 fits
[CV 1/5] END final_estimator__alpha=0.0001, final_estimator__fit_intercept=True, final_estimator__l1_ratio=0.1, final_estimator__max_iter=1000, final_estimator__random_state=42, final_estimator__selection=cyclic, final_estimator__tol=0.0001;, score=-2.186 total time=  16.3s
[CV 2/5] END final_estimator__alpha=0.0001, final_estimator__fit_intercept=True, final_estimator__l1_ratio=0.1, final_estimat

c:\Users\User\.conda\envs\energy\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.927e+04, tolerance: 1.615e+02
  model = cd_fast.enet_coordinate_descent(


[CV 1/5] END final_estimator__alpha=0.0001, final_estimator__fit_intercept=False, final_estimator__l1_ratio=0.1, final_estimator__max_iter=1000, final_estimator__random_state=42, final_estimator__selection=cyclic, final_estimator__tol=0.0001;, score=-2.181 total time=  20.3s


c:\Users\User\.conda\envs\energy\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.796e+04, tolerance: 1.598e+02
  model = cd_fast.enet_coordinate_descent(


[CV 2/5] END final_estimator__alpha=0.0001, final_estimator__fit_intercept=False, final_estimator__l1_ratio=0.1, final_estimator__max_iter=1000, final_estimator__random_state=42, final_estimator__selection=cyclic, final_estimator__tol=0.0001;, score=-2.329 total time=  21.6s


...

c:\Users\User\.conda\envs\energy\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.773e+03, tolerance: 1.614e+03
  model = cd_fast.enet_coordinate_descent(


[CV 4/5] END final_estimator__alpha=0.01, final_estimator__fit_intercept=False, final_estimator__l1_ratio=0.5, final_estimator__max_iter=1000, final_estimator__random_state=42, final_estimator__selection=cyclic, final_estimator__tol=0.001;, score=-2.146 total time=  20.7s


c:\Users\User\.conda\envs\energy\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.604e+03, tolerance: 1.609e+03
  model = cd_fast.enet_coordinate_descent(


[CV 5/5] END final_estimator__alpha=0.01, final_estimator__fit_intercept=False, final_estimator__l1_ratio=0.5, final_estimator__max_iter=1000, final_estimator__random_state=42, final_estimator__selection=cyclic, final_estimator__tol=0.001;, score=-2.281 total time=  24.0s
[CV 1/5] END final_estimator__alpha=0.01, final_estimator__fit_intercept=False, final_estimator__l1_ratio=0.5, final_estimator__max_iter=1000, final_estimator__random_state=42, final_estimator__selection=cyclic, final_estimator__tol=0.01;, score=-2.191 total time=  27.0s
[CV 2/5] END final_estimator__alpha=0.01, final_estimator__fit_intercept=False, final_estimator__l1_ratio=0.5, final_estimator__max_iter=1000, final_estimator__random_state=42, final_estimator__selection=cyclic, final_estimator__tol=0.01;, score=-2.340 total time=  17.6s
[CV 3/5] END final_estimator__alpha=0.01, final_estimator__fit_intercept=False, final_estimator__l1_ratio=0.5, final_estimator__max_iter=1000, final_estimator__random_state=42, final_

KeyboardInterrupt: 

In [20]:
# print dtypes
print(X.dtypes)

source_1_temperature     float32
source_2_temperature     float32
source_3_temperature     float32
source_4_temperature     float32
mean_room_temperature    float32
outside_temperature      float32
hour_sin                 float32
hour_cos                 float32
holiday_1                   bool
work_hours_1                bool
dtype: object
